In [27]:
include("src/init.jl")
include("src/fct.jl")
data_type = "cal"
dataset = "v07.01"

"v07.01"

## sort_data_all_01-new.jl
For several runs

In [16]:
meta_key_file = "datasets/run0053-run0114-cal-analysis.txt";
meta_keys = CSV.read(meta_key_file);
channels = 0:1:36
event_step = Int(1e4)

current_dir = pwd()
ddir = "/remote/ceph/group/gerda/data/phase2/blind/" * dataset * "/gen/"
cd(ddir)
filenames1 = []
filenames4 = []
for meta_key in meta_keys[1]
    filename1 = ddir * glob("tier1/ged/" * data_type * "/" * split(meta_key, "-")[2] * "/" * meta_key * "*.root")[1]
    filename4 = ddir * glob("tier4/all/" * data_type * "/" * split(meta_key, "-")[2] * "/" * meta_key * "*.root")[1]
    push!(filenames1, filename1)
    push!(filenames4, filename4)
end
cd(current_dir)

if data_type == "phy"
    base_path = "pulses/data/raw_" * dataset * "/"
elseif data_type == "cal"
    base_path = "pulses/calib/raw_" * dataset * "/"
end
log_file = base_path * "log.json"
if !isdir(base_path)
    mkpath(base_path)
end
if !isfile(log_file)
    open(log_file, "w") do f
        JSON.print(f, Dict(), 4)
    end
    global log = Dict()
else
    global log = JSON.parsefile(log_file)
end;

for i in eachindex(filenames4)
    start_t = now()
    run = parse(Int64, split(split(filenames4[i], "gerda-run")[2], "-")[1])
    filename1 = filenames1[i]
    filename4 = filenames4[i]
    
    @info("Run " * string(run) * " | file " * string(i) * " of total " * string(length(filenames4)))
    log = JSON.parsefile(log_file)
    if length(keys(log)) >= 1
        estimation = 0
        for k in keys(log)
            estimation += log[k]
        end
        estimation /= length(keys(log))
        fileinfo = stat(filename4)
        est_t = round(fileinfo.size * estimation / 60, digits=1)
        @info("Estimated time for this file: " * string(est_t) * " min")
        total_filesize = 0
        for f in i:length(filenames4)
            total_filesize += stat(filenames4[f]).size
        end
        est_t = total_filesize * estimation / 60
        unit = " s"
        if est_t/60/60/24 > 1
            est_t /= 60*60*24
            unit = " d"
        elseif est_t/60/60 > 1
            est_t /= 60*60
            unit = " h"
        elseif est_t/60 > 1
            est_t /= 60
            unit = " min"
        end
        @info("Estimated time till completion: " * string(round(est_t, digits=1)) * unit)
    end
    @info("------------------------------")
    

    if true#!(filename4 in keys(JSON.parsefile(log_file)))
        @info("Load Tier4")
        @time tier4 = Table(TFile(filename4)["tier4"]);
        temp_E = sum.(tier4.energy[:])

        index_1 = findall(x->x == 1, tier4.multiplicity[:])
        index_2 = findall(y->y > 300, temp_E[index_1])
        index_3 = findall(x->x == 0, tier4.isBL[index_1[index_2]])
        index_4 = findall(x->x == 0, tier4.isTP[index_1[index_2[index_3]]])
        filtered_index = index_1[index_2[index_3[index_4]]]
        if length(filtered_index) > 0
            @info("Number of events: " * string(length(temp_E)))
            @info("Number of events after filter: " * string(length(filtered_index)))
            steps = []
            s = 1
            while s <= length(filtered_index)
                a = s
                b = s + event_step - 1 <= length(filtered_index) ? s+event_step-1 : length(filtered_index)
#                 println(string(filtered_index[a]) * " - " * string(filtered_index[b]))
                push!(steps, [a,b])
                s += event_step
            end
            return steps
            for s in steps
                result = Table( energy       = [],
                        run          = [],
                        channel      = [],
                        AEvetoed     = [],
                        datasetID    = [],
                        AEclassifier = [],
                        waveform     = [])
                IJulia.clear_output(true)
                run_str = split(basename(filename4), "-")[2]
                @info("Run " * run_str * " | file " * string(i) * " of total " * string(length(filenames4)))
                @info("Step " * string(findfirst(x->x == s, steps)) * " of " * string(length(steps)))
                file = base_path * run_str * "/"
                if !isdir(file)
                    mkpath(file)
                end
                file *= basename(filename4) * lpad(findfirst(x->x == s, steps), 4, "0") * ".h5"
                if !isfile(file)
                    
                    tmp = filtered_index[s[1]:s[2]]
                    @info("Event " * string(tmp[1]) * " until " * string(tmp[end]))
                    tier4 = Table(TFile(filename4)["tier4"])[tmp];
                    @info("Load Tier1")
                    @time treeTier1 = TFile(filename1)["MGTree"].event;
                    @info("Apply first filter to Tier1 & Tier4")
                    @time waveforms = TypedTables.Table(raw2mgtevent.(treeTier1[tmp])).fAuxWaveforms
        #             waveforms = waveforms[tmp]
                    if length(tmp) > 0
                        run_str = split(basename(filename4), "-")[2]
                        run = parse(Int64, split(run_str, "run")[2])

                        pro = Progress(length(channels), dt=0.5,
                            barglyphs=BarGlyphs('|','█', ['▁' ,'▂' ,'▃' ,'▄' ,'▅' ,'▆', '▇'],' ','|',),
                            barlen=10);
                        for ch in channels

                            tmp = findall(y->y[ch + 1] != 0, tier4.energy)
                            if length(tmp) > 0
                                waveform = []
                                for i in tmp
                                    if data_type == "phy"
                                        push!(waveform, waveforms[i][ch + 1].wf)
                                    else
                                        push!(waveform, waveforms[i][1].wf)
                                    end
                                end

                                AEvetoed     = []
                                datasetID    = []
                                AEclassifier = []
                                for i in tmp
                                    push!(AEvetoed,     tier4[i].isAoEvetoed[ch+1])
                                    push!(datasetID,    tier4[i].datasetID[ch+1])
                                    push!(AEclassifier, tier4[i].AoEclassifier[ch+1])
                                end

                                append!(result, Table(  energy       = sum.(tier4.energy[tmp]),
                                                        run          = Int.(zeros(length(tmp)) .+ run),
                                                        channel      = Int.(zeros(length(tmp)) .+ ch),
                                                        AEvetoed     = AEvetoed,
                                                        datasetID    = datasetID,
                                                        AEclassifier = AEclassifier,
                                                        waveform     = StructArray{RDWaveform}(Array{RDWaveform,1}(waveform))))
                            end
                            next!(pro)
                        end
                    end
                    if size(result,1) > 0
                        file = base_path * run_str * "/"
                        file *= basename(filename4) * lpad(findfirst(x->x == s, steps), 4, "0") * ".h5"
                        HDF5.h5open(file, "w") do h5f
                            LegendHDF5IO.writedata( h5f, "data", Table( energy       = float.(result.energy),
                                                                        run          = Int.(result.run),
                                                                        channel      = Int.(result.channel),
                                                                        AEvetoed     = Int.(result.AEvetoed),
                                                                        datasetID    = Int.(result.datasetID),
                                                                        AEclassifier = float.(result.AEclassifier),
                                                                        waveform     = StructArray{RDWaveform}(Array{RDWaveform,1}(result.waveform))))
                        end
                    end
                end
            end
            log = JSON.parsefile(log_file)
            dt = now() - start_t
            fileinfo = stat(filenames4[1])
            fileinfo.size
            log[filename4] = (dt.value / 1000) / fileinfo.size
            open(log_file, "w") do f
                JSON.print(f, log, 4)
            end
        end
    end
    IJulia.clear_output(true)
#     Base.run(`clear`)
    log = JSON.parsefile(log_file)
    @info(string(length(log)) * " of " * string(length(filenames4)) * " done!")
end

┌ Info: Run 53 | file 1 of total 4344
└ @ Main In[16]:43
┌ Info: Estimated time for this file: 221.5 min
└ @ Main In[16]:53
┌ Info: Estimated time till completion: 1.9 d
└ @ Main In[16]:70
┌ Info: ------------------------------
└ @ Main In[16]:72
┌ Info: Load Tier4
└ @ Main In[16]:76


 25.570073 seconds (4.95 M allocations: 3.452 GiB, 1.72% gc time)


┌ Info: Number of events: 833103
└ @ Main In[16]:86
┌ Info: Number of events after filter: 655657
└ @ Main In[16]:87


66-element Array{Any,1}:
 [1, 10000]
 [10001, 20000]
 [20001, 30000]
 [30001, 40000]
 [40001, 50000]
 [50001, 60000]
 [60001, 70000]
 [70001, 80000]
 [80001, 90000]
 [90001, 100000]
 [100001, 110000]
 [110001, 120000]
 [120001, 130000]
 ⋮
 [540001, 550000]
 [550001, 560000]
 [560001, 570000]
 [570001, 580000]
 [580001, 590000]
 [590001, 600000]
 [600001, 610000]
 [610001, 620000]
 [620001, 630000]
 [630001, 640000]
 [640001, 650000]
 [650001, 655657]

In [8]:
meta_key_file = "datasets/run0053-run0114-cal-analysis.txt";
meta_keys = CSV.read(meta_key_file);
channels = 0:1:36
event_step = Int(1e4)

current_dir = pwd()
ddir = "/remote/ceph/group/gerda/data/phase2/blind/" * dataset * "/gen/"
cd(ddir)
filenames1 = []
filenames4 = []
for meta_key in meta_keys[1]
    filename1 = ddir * glob("tier1/ged/" * data_type * "/" * split(meta_key, "-")[2] * "/" * meta_key * "*.root")[1]
    filename4 = ddir * glob("tier4/all/" * data_type * "/" * split(meta_key, "-")[2] * "/" * meta_key * "*.root")[1]
    push!(filenames1, filename1)
    push!(filenames4, filename4)
end
cd(current_dir)
if data_type == "phy"
    base_path = "pulses/data/raw_" * dataset * "/"
elseif data_type == "cal"
    base_path = "pulses/calib/raw_" * dataset * "/"
end
log_file = base_path * "log.json"
if !isdir(base_path)
    mkpath(base_path)
end
if !isfile(log_file)
    open(log_file, "w") do f
        JSON.print(f, Dict(), 4)
    end
    global log = Dict()
else
    global log = JSON.parsefile(log_file)
end;

In [9]:
(stat(filenames1[1]).size)/1024/1024
process_speed = ((stat(filenames1[1]).size)/1024/1024) / (242 * 60)
total_size = 0
@time (for f in filenames1
    total_size += (stat(f).size)/1024/1024
end)
(total_size / process_speed) / 3600 / 24

  0.030328 seconds (42.46 k allocations: 2.506 MiB)


99.29626197055194

In [48]:
@time treeTier1 = TFile(filenames1[1])["MGTree"].event;
@time waveforms = TypedTables.Table(raw2mgtevent.(treeTier1[1:1000])).fAuxWaveforms;

  0.034966 seconds (136 allocations: 5.328 KiB)
  0.166404 seconds (150.24 k allocations: 62.017 MiB, 5.46% gc time)


In [4]:
begin
   lock(a)
   try
       use(a)
   finally
       unlock(a)
   end
end

In [8]:
Threads.@threads for f in filenames1[1:3]
    println(string(Threads.threadid()))
    lock(treeTier1) do 
        treeTier1 = TFile(f)["MGTree"].event;
        @time waveforms = TypedTables.Table(raw2mgtevent.(treeTier1[1:1000])).fAuxWaveforms
        waveform = []
        for wf in waveforms
            push!(waveform, wf.wf[1])
        end
        HDF5.h5open("testing/" * basename(f) * ".h5", "w") do h5f
            LegendHDF5IO.writedata(h5f, "data", Table(waveform = StructArray{RDWaveform}(Array{RDWaveform,1}(waveform))))
        end
        waveforms = nothing
    end
end

1
2
3


LoadError: TaskFailedException:
MethodError: no method matching lock(::var"#39#42"{String}, ::Nothing)
Closest candidates are:
  lock(::Any, !Matched::Base.GenericCondition) at condition.jl:78
  lock(::Any, !Matched::Base.AbstractLock) at lock.jl:158
  lock(::Any, !Matched::WeakKeyDict) at weakkeydict.jl:76
Stacktrace:
 [1] macro expansion at ./In[8]:3 [inlined]
 [2] (::var"#404#threadsfor_fun#41"{Array{Any,1}})(::Bool) at ./threadingconstructs.jl:81
 [3] (::var"#404#threadsfor_fun#41"{Array{Any,1}})() at ./threadingconstructs.jl:48

In [4]:
steps = [1:1:1000, 1001:1:2000, 2001:1:3000]

3-element Array{StepRange{Int64,Int64},1}:
 1:1:1000
 1001:1:2000
 2001:1:3000

In [5]:
f = filenames1[1]
treeTier1 = TFile(f)["MGTree"].event;
Threads.@threads for s in steps
    lock(treeTier1) do
        local waveforms = TypedTables.Table(raw2mgtevent.(treeTier1[s])).fAuxWaveforms
        local waveform = []
        for wf in waveforms
            push!(waveform, wf.wf[1])
        end
        HDF5.h5open("testing/" * basename(f) * string(findfirst(x->x==s, steps)) * ".h5", "w") do h5f
            LegendHDF5IO.writedata(h5f, "data", Table(waveform = StructArray{RDWaveform}(Array{RDWaveform,1}(waveform))))
        end
    end
end

LoadError: TaskFailedException:
MethodError: no method matching lock(::var"#32#36"{StepRange{Int64,Int64}}, ::TBranch)
Closest candidates are:
  lock(::Any, !Matched::Base.GenericCondition) at condition.jl:78
  lock(::Any, !Matched::Base.AbstractLock) at lock.jl:158
  lock(::Any, !Matched::WeakKeyDict) at weakkeydict.jl:76
Stacktrace:
 [1] macro expansion at ./In[5]:4 [inlined]
 [2] (::var"#343#threadsfor_fun#35"{Array{StepRange{Int64,Int64},1}})(::Bool) at ./threadingconstructs.jl:81
 [3] (::var"#343#threadsfor_fun#35"{Array{StepRange{Int64,Int64},1}})() at ./threadingconstructs.jl:48

In [16]:
using Distributed; 
addprocs(4);
nprocs()

9

In [26]:
filenames1[1]

"/remote/ceph/group/gerda/data/phase2/blind/v07.01/gen/tier1/ged/cal/run0053/gerda-run0053-20151223T105042Z-cal-ged-tier1.root"

In [25]:
f = filenames1[1]
treeTier1 = TFile(f)["MGTree"].event;
@distributed for s in steps
    @time waveforms = TypedTables.Table(raw2mgtevent.(treeTier1[s])).fAuxWaveforms
end

Task (failed) @0x00007fdd23b24280
InterruptException:
sigatomic_end at ./c.jl:425 [inlined]
disable_sigint at ./c.jl:448 [inlined]
__pycall! at /user/.julia/packages/PyCall/BcTLp/src/pyfncall.jl:42 [inlined]
_pycall!(::PyCall.PyObject, ::PyCall.PyObject, ::Tuple{}, ::Int64, ::PyCall.PyObject) at /user/.julia/packages/PyCall/BcTLp/src/pyfncall.jl:29
_pycall!(::PyCall.PyObject, ::PyCall.PyObject, ::Tuple{}, ::Base.Iterators.Pairs{Symbol,Int64,Tuple{Symbol,Symbol},NamedTuple{(:entrystart, :entrystop),Tuple{Int64,Int64}}}) at /user/.julia/packages/PyCall/BcTLp/src/pyfncall.jl:11
#_#117 at /user/.julia/packages/PyCall/BcTLp/src/pyfncall.jl:86 [inlined]
getindex(::TBranch, ::UnitRange{Int64}) at /user/.julia/packages/UpROOT/pN5xO/src/ttree.jl:127
getindex(::TBranch, ::Int64) at /user/.julia/packages/UpROOT/pN5xO/src/ttree.jl:131
hash(::TBranch, ::UInt64) at ./abstractarray.jl:2297
hash(::TBranch) at ./hashing.jl:18
serialize_global_from_main(::Distributed.ClusterSerializer{Sockets.TCPSocket}

In [ ]:
    local waveform = []
    for wf in waveforms
        push!(waveform, wf.wf[1])
    end
#     HDF5.h5open("testing/" * basename(f) * string(findfirst(x->x==s, steps)) * ".h5", "w") do h5f
#         LegendHDF5IO.writedata(h5f, "data", Table(waveform = StructArray{RDWaveform}(Array{RDWaveform,1}(waveform))))
#     end

# Calibration & splitting into detectors (Sep 2020)

In [ ]:
if data_type == "phy"
    plots_base_path = "pulses/data/raw_" * dataset * "/plots/"
    base_path_raw   = "pulses/data/raw_" * dataset * "/"
    base_path       = "pulses/data/" * dataset * "/"
elseif data_type == "cal"
    plots_base_path = "pulses/calib/raw_" * dataset * "/plots/"
    base_path_raw   = "pulses/calib/raw_" * dataset * "/"
    base_path       = "pulses/calib/" * dataset * "/"
end
number_of_pulses = 5e3
bl_range = 1:1:200
cal = JSON.parsefile("data_pulses/FEP_pulses/calibration/cal.json");

In [ ]:
files = glob(base_path_raw * "*/*.h5");
data_cal = Dict()
for ch in 0:1:36
    data_cal[string(ch)] = Dict()
    data_cal[string(ch)]["data"] = Table(energy = [], run = [], channel = [], AoEvetoed = [], datasetID = [], AoEclassifier = [], A = [], E = [], waveform = [])
    data_cal[string(ch)]["counter"] = 1
end
pro = Progress(length(files), dt=0.5,
                barglyphs=BarGlyphs('|','█', ['▁' ,'▂' ,'▃' ,'▄' ,'▅' ,'▆', '▇'],' ','|',),
                barlen=10);
for file in files
    if stat(file).size > 1000
        data = HDF5.h5open(file, "r") do h5f
            LegendHDF5IO.readdata(h5f, "data")
        end;
        for ch in unique(data.channel)
            ch_str = lpad(ch, 2, "0");
            temp = data |> @filter(_.channel == ch) |> Table
            cal_factor = cal[string(ch)]["value"]
            waveforms = []
            A = []
            E = []
            for wf in temp.waveform
                pulse   = -1 .*float.(wf.value)
                pulse .-= sum(pulse[bl_range])/length(bl_range)
                pulse .*= cal_factor
                push!(A, maximum(multi_mwa(diff(pulse),5,3)))
                push!(E, maximum(multi_mwa(diff(pulse),201,6)))
                push!(waveforms, RDWaveform(wf.time, pulse))
            end
            
            append!(data_cal[string(ch)]["data"], Table(energy   = temp.energy,
                                                        run      = temp.run,
                                                        channel  = temp.channel,
                                                        AoEvetoed= temp.AEvetoed,
                                                        datasetID= temp.datasetID,
                                                        AoEclassifier = temp.AEclassifier,
                                                        A        = A,
                                                        E        = E,
                                                        waveform = waveforms))
            if size(data_cal[string(ch)]["data"],1) >= number_of_pulses
                filename = base_path * channel_to_name[ch] * "-" * ch_str * "/"
                !isdir(filename) ? mkpath(filename) : "path exists"
                filename *= string(data_cal[string(ch)]["counter"]) * ".h5"
                data_cal[string(ch)]["counter"] += 1
                HDF5.h5open(filename, "w") do h5f
                    LegendHDF5IO.writedata( h5f, "data", Table( energy       = float.(data_cal[string(ch)]["data"].energy),
                                                                run          = Int.(data_cal[string(ch)]["data"].run),
                                                                channel      = Int.(data_cal[string(ch)]["data"].channel),
                                                                AoEvetoed     = Int.(data_cal[string(ch)]["data"].AoEvetoed),
                                                                datasetID    = Int.(data_cal[string(ch)]["data"].datasetID),
                                                                AoEclassifier= float.(data_cal[string(ch)]["data"].AoEclassifier),
                                                                A            = float.(data_cal[string(ch)]["data"].A),
                                                                E            = float.(data_cal[string(ch)]["data"].E),
                                                                waveform     = StructArray{RDWaveform}(Array{RDWaveform,1}(data_cal[string(ch)]["data"].waveform))))
                end
                data_cal[string(ch)]["data"] = Table(energy = [], run = [], channel = [], AoEvetoed = [], datasetID = [], AoEclassifier = [], A = [], E = [], waveform = [])
            end
        end
    end
#     IJulia.clear_output(true)
#     @info(string(findfirst(x->x == file, files)) * " of " * string(length(files)))
#     for ch in keys(data_cal)
#         ch_str = lpad(parse(Int64, ch), 2, "0");
#         println("Ch" * ch_str * " | " * string(size(data_cal[ch]["data"],1)))
#     end
    next!(pro)
end
for ch in 0:1:36
    ch_str = lpad(ch, 2, "0");
    if size(data_cal[string(ch)]["data"],1) > 0
        filename = base_path * channel_to_name[ch] * "-" * ch_str * "/"
        !isdir(filename) ? mkpath(filename) : "path exists"
        filename *= string(data_cal[string(ch)]["counter"]) * ".h5"
        data_cal[string(ch)]["counter"] += 1
        HDF5.h5open(filename, "w") do h5f
            LegendHDF5IO.writedata( h5f, "data", Table( energy       = float.(data_cal[string(ch)]["data"].energy),
                                                        run          = Int.(data_cal[string(ch)]["data"].run),
                                                        channel      = Int.(data_cal[string(ch)]["data"].channel),
                                                        AoEvetoed     = Int.(data_cal[string(ch)]["data"].AoEvetoed),
                                                        datasetID    = float.(data_cal[string(ch)]["data"].datasetID),
                                                        AoEclassifier= float.(data_cal[string(ch)]["data"].AoEclassifier),
                                                        A            = float.(data_cal[string(ch)]["data"].A),
                                                        E            = float.(data_cal[string(ch)]["data"].E),
                                                        waveform     = StructArray{RDWaveform}(Array{RDWaveform,1}(data_cal[string(ch)]["data"].waveform))))
        end
    end
end

# Get calibration constant for each run